# Analysis

In [106]:
import json

with open('diffs.json', 'r') as f:
    diffs = json.load(f)

with open('codes.json', 'r') as cf:
    codes = json.load(cf)

## Dataset

In [107]:
TOTAL = 538
retrieved = len(diffs)
with_saving = len([d for d in diffs if 'saving' in d['existing_codes']])
print('retrieved commits:', f'{retrieved}/{TOTAL}', f'({retrieved/TOTAL*100:.2f}%)')
print('originally coded with `saving`:', f'{with_saving}/{retrieved}', f'({with_saving/retrieved*100:.2f}%)')

n_processed = sum(len(d['codes']) != 0 for d in diffs)
print('processed commits:', f'{n_processed}/{retrieved}', f'({n_processed/retrieved*100:.2f}%)')
n_processed_with_saving = sum(len(d['codes']) != 0 and 'saving' in d['existing_codes'] for d in diffs)
print('processed commits coded with `saving`:', f'{n_processed_with_saving}/{with_saving}', f'({n_processed_with_saving/with_saving*100:.2f}%)')

avg_files = sum(len(d['files']) for d in diffs) / len(diffs)
print('files affected (avg):', f'{avg_files:.2f}')

retrieved commits: 499/538 (92.75%)
originally coded with `saving`: 342/499 (68.54%)
processed commits: 408/499 (81.76%)
processed commits coded with `saving`: 312/342 (91.23%)
files affected (avg): 4.42


## Codes

In [108]:
import polars as pl

df = pl.DataFrame(codes) \
        .with_columns(count=sum(pl.col('name').is_in(d['codes']) for d in diffs)) \
        .with_columns(percentage=pl.col('count') / n_processed * 100) \
        .sort(by=pl.col('count'), descending=True)

print(df)

shape: (106, 4)
┌───────────────────────────────────┬───────────────────────────────────┬───────┬────────────┐
│ name                              ┆ description                       ┆ count ┆ percentage │
│ ---                               ┆ ---                               ┆ ---   ┆ ---        │
│ str                               ┆ str                               ┆ i32   ┆ f64        │
╞═══════════════════════════════════╪═══════════════════════════════════╪═══════╪════════════╡
│ aws                               ┆ Specific to Amazon Web Services   ┆ 298   ┆ 73.039216  │
│ not_relevant                      ┆ The commit contents do not refle… ┆ 137   ┆ 33.578431  │
│ cheaper_instance                  ┆ Use a cheaper compute instance    ┆ 68    ┆ 16.666667  │
│ gcp                               ┆ Specific to Google Cloud Platfor… ┆ 51    ┆ 12.5       │
│ azure                             ┆ Specific to Azure                 ┆ 41    ┆ 10.04902   │
│ …                               

### Cloud-specific Codes

In [109]:
clouds = {'digitalocean', 'aws', 'gcp', 'azure', 'alicloud', 'scaleway', 'hcloud', 'ibm', 'oracle', 'ovh'}
print(df.filter(pl.col('name').is_in(clouds)).drop('description'))

shape: (10, 3)
┌──────────────┬───────┬────────────┐
│ name         ┆ count ┆ percentage │
│ ---          ┆ ---   ┆ ---        │
│ str          ┆ i32   ┆ f64        │
╞══════════════╪═══════╪════════════╡
│ aws          ┆ 298   ┆ 73.039216  │
│ gcp          ┆ 51    ┆ 12.5       │
│ azure        ┆ 41    ┆ 10.04902   │
│ digitalocean ┆ 10    ┆ 2.45098    │
│ scaleway     ┆ 2     ┆ 0.490196   │
│ oracle       ┆ 2     ┆ 0.490196   │
│ alicloud     ┆ 1     ┆ 0.245098   │
│ hcloud       ┆ 1     ┆ 0.245098   │
│ ibm          ┆ 1     ┆ 0.245098   │
│ ovh          ┆ 1     ┆ 0.245098   │
└──────────────┴───────┴────────────┘


### Undefined codes

List codes which are attached to commits but not defined in `codes.json`.

In [110]:
listed_codes = set(c['name'] for c in codes)
for d in diffs:
    for c in d['codes']:
        if c not in listed_codes:
            print(f'{c} is not listed')